# Documentação do Processo de Pipeline de Dados

Este notebook documenta o fluxo completo do pipeline de dados desde a ingestão até a camada gold.


## Visão Geral do Processo

O pipeline de dados é dividido em 3 camadas principais:

### 1. SOURCE TO BRONZE (Ingestão)
- **3 ingestões de dados:**
  - `ingestion_claim.sql`: Ingestão de dados de claims
  - `ingestion_customers.sql`: Ingestão de dados de customers
  - `ingestion_policies.sql`: Ingestão de dados de policies
- Dados são carregados do volume para tabelas na camada bronze
- Formato: CSV
- Destino: `${catalog}.${schema_bronze}`

### 2. BRONZE TO SILVER (Transformação e Enriquecimento)
- **01_deduplicate_claims.sql**: Deduplicação de claims
  - Remove duplicatas mantendo apenas o registro mais recente por claim_no
  - Usa ROW_NUMBER() com PARTITION BY claim_no ORDER BY claim_date DESC
  - Cria tabela: `claims_dedup`
  
- **02_join_tables_create.sql**: Join de tabelas
  - Combina claims_dedup com policies e customers
  - Cria tabela enriquecida: `claims_enriched`
  - Inclui informações de políticas e clientes

### 3. SILVER TO GOLD (Métricas e Agregações)
- **03_join_tables_metrics.sql**: Cálculo de métricas
  - Gera métricas agregadas da tabela claims_enriched
  - Conta registros, claims únicos, policies únicas, customers únicos


## Diagrama do Fluxo do Processo

```mermaid
graph TD
    A[Volume Landing Zone<br/>CSV Files] --> B1[Ingestão Claims<br/>ingestion_claim.sql]
    A --> B2[Ingestão Customers<br/>ingestion_customers.sql]
    A --> B3[Ingestão Policies<br/>ingestion_policies.sql]
    
    B1 --> C1[Bronze: claims]
    B2 --> C2[Bronze: customers]
    B3 --> C3[Bronze: policies]
    
    C1 --> D[Deduplicação<br/>01_deduplicate_claims.sql]
    D --> E[Silver: claims_dedup]
    
    E --> F[Join de Tabelas<br/>02_join_tables_create.sql]
    C2 --> F
    C3 --> F
    
    F --> G[Silver: claims_enriched]
    
    G --> H[Cálculo de Métricas<br/>03_join_tables_metrics.sql]
    H --> I[Gold: Métricas]
    
    style A fill:#e1f5ff
    style C1 fill:#fff4e1
    style C2 fill:#fff4e1
    style C3 fill:#fff4e1
    style E fill:#e8f5e9
    style G fill:#e8f5e9
    style I fill:#f3e5f5
```


## Detalhamento das Etapas


### Etapa 1: Source to Bronze (Ingestão)

**Objetivo**: Carregar dados brutos do volume para a camada bronze

**Notebooks:**
- `source_to_bronze/ingestion_claim.sql`
- `source_to_bronze/ingestion_customers.sql`
- `source_to_bronze/ingestion_policies.sql`

**Características:**
- Processa arquivos CSV do volume `/Volumes/${catalog}/00_landing/sql_server/`
- Usa `read_files()` para leitura dos arquivos
- Cria tabelas na camada bronze sem transformações
- Permite reprocessamento completo (DROP TABLE IF EXISTS)


### Etapa 2: Bronze to Silver (Transformação)

**Objetivo**: Limpar, deduplicar e enriquecer os dados

#### 2.1 Deduplicação (`01_deduplicate_claims.sql`)
- Remove duplicatas da tabela claims
- Mantém apenas o registro mais recente por `claim_no`
- Ordena por `claim_date DESC NULLS LAST`
- Cria tabela `claims_dedup` na camada silver

#### 2.2 Join de Tabelas (`02_join_tables_create.sql`)
- Combina `claims_dedup` com `policies` e `customers`
- Realiza INNER JOINs para enriquecer os dados
- Cria tabela `claims_enriched` com todas as informações consolidadas
- Adiciona timestamp `processed_at` para rastreabilidade


### Etapa 3: Silver to Gold (Métricas)

**Objetivo**: Calcular métricas e agregações para análise

**Notebook**: `silver_to_gold/03_join_tables_metrics.sql`

**Métricas calculadas:**
- Total de registros enriquecidos
- Número de claims únicos
- Número de policies únicas
- Número de customers únicos
- Mensagem de conclusão do processo


## Variáveis Utilizadas

### Variáveis Obrigatórias do Pipeline

O pipeline utiliza as seguintes variáveis que **DEVEM** ser configuradas antes da execução:

| Variável | Descrição | Exemplo | Onde é Usada |
|----------|-----------|---------|--------------|
| `${catalog}` | Nome do catálogo Unity Catalog | `smart_claims_dev` | Todas as etapas |
| `${schema_bronze}` | Nome do schema da camada bronze | `01_bronze` | Source to Bronze, Bronze to Silver |
| `${schema_silver}` | Nome do schema da camada silver | `02_silver` | Bronze to Silver, Silver to Gold |
| `${schema_gold}` | Nome do schema da camada gold | `03_gold` | Silver to Gold |

### Exemplo de Valores Recomendados

```python
catalog = "smart_claims_dev"
schema_bronze = "01_bronze"
schema_silver = "02_silver"
schema_gold = "03_gold"
```

### Como Configurar as Variáveis

#### Opção 1: Via Databricks Widgets (Recomendado para Notebooks)

Adicione no início de cada notebook:

```python
dbutils.widgets.text("catalog", "smart_claims_dev", "Catálogo")
dbutils.widgets.text("schema_bronze", "01_bronze", "Schema Bronze")
dbutils.widgets.text("schema_silver", "02_silver", "Schema Silver")
dbutils.widgets.text("schema_gold", "03_gold", "Schema Gold")

# Recuperar valores
catalog = dbutils.widgets.get("catalog")
schema_bronze = dbutils.widgets.get("schema_bronze")
schema_silver = dbutils.widgets.get("schema_silver")
schema_gold = dbutils.widgets.get("schema_gold")
```

#### Opção 2: Via Variáveis de Ambiente

Configure antes de executar os notebooks:

```bash
export CATALOG=smart_claims_dev
export SCHEMA_BRONZE=01_bronze
export SCHEMA_SILVER=02_silver
export SCHEMA_GOLD=03_gold
```

#### Opção 3: Via Databricks Workflows (Recomendado para Produção)

Configure as variáveis nas **Task Parameters** de cada task:

1. Acesse o Job/Workflow no Databricks
2. Edite cada task
3. Na seção **Parameters**, adicione:
   - `catalog`: `smart_claims_dev`
   - `schema_bronze`: `01_bronze`
   - `schema_silver`: `02_silver`
   - `schema_gold`: `03_gold`

### Verificação das Variáveis

Antes de executar o pipeline, verifique se as variáveis estão configuradas corretamente:

```sql
-- Verificar catálogo
SHOW CATALOGS LIKE '${catalog}';

-- Verificar schemas
SHOW SCHEMAS IN ${catalog};

-- Verificar se os schemas existem
SHOW SCHEMAS IN ${catalog} LIKE '${schema_bronze}';
SHOW SCHEMAS IN ${catalog} LIKE '${schema_silver}';
SHOW SCHEMAS IN ${catalog} LIKE '${schema_gold}';
```

### Importante

⚠️ **Atenção**: Certifique-se de que:
- O catálogo `${catalog}` já existe (criado via `01_create_catalog_and_schemas.ipynb`)
- Os schemas `${schema_bronze}`, `${schema_silver}` e `${schema_gold}` já existem
- Você tem permissões adequadas para criar tabelas nos schemas
- O volume `/Volumes/${catalog}/00_landing/sql_server/` existe e contém os arquivos CSV


### Exemplo Prático: Configuração em Notebook SQL

Para notebooks SQL no Databricks, você pode usar widgets Python em uma célula separada:

**Célula Python (executar primeiro):**
```python
# Configurar widgets
dbutils.widgets.text("catalog", "smart_claims_dev", "Catálogo")
dbutils.widgets.text("schema_bronze", "01_bronze", "Schema Bronze")
dbutils.widgets.text("schema_silver", "02_silver", "Schema Silver")
dbutils.widgets.text("schema_gold", "03_gold", "Schema Gold")
```

**Células SQL (usar as variáveis):**
```sql
-- As variáveis ${catalog}, ${schema_bronze}, etc. estarão disponíveis
USE CATALOG ${catalog};
USE SCHEMA ${schema_bronze};

-- Exemplo de uso
SELECT * FROM ${catalog}.${schema_bronze}.claims LIMIT 10;
```

### Exemplo Prático: Configuração em Databricks Workflows

Ao criar um Workflow, configure os parâmetros em cada task:

**Task: ingestion_claim**
- Parameters:
  - `catalog`: `smart_claims_dev`
  - `schema_bronze`: `01_bronze`

**Task: 01_deduplicate_claims**
- Parameters:
  - `catalog`: `smart_claims_dev`
  - `schema_bronze`: `01_bronze`
  - `schema_silver`: `02_silver`

**Task: 02_join_tables_create**
- Parameters:
  - `catalog`: `smart_claims_dev`
  - `schema_bronze`: `01_bronze`
  - `schema_silver`: `02_silver`

**Task: 03_join_tables_metrics**
- Parameters:
  - `catalog`: `smart_claims_dev`
  - `schema_silver`: `02_silver`
  - `schema_gold`: `03_gold`


## Ordem de Execução Recomendada

### Sequência de Execução

1. **Ingestão (Source to Bronze)**
   - Executar `ingestion_claim.sql`
   - Executar `ingestion_customers.sql`
   - Executar `ingestion_policies.sql`
   - *Ordem pode ser paralela, mas todas devem completar antes do próximo passo*

2. **Transformação (Bronze to Silver)**
   - Executar `01_deduplicate_claims.sql` (depende de bronze.claims)
   - Executar `02_join_tables_create.sql` (depende de claims_dedup, policies, customers)

3. **Métricas (Silver to Gold)**
   - Executar `silver_to_gold/03_join_tables_metrics.sql` (depende de claims_enriched)

**Nota**: Esta sequência pode ser automatizada usando Databricks Workflows com dependências entre tasks.
